In [1]:
from pathlib import Path
import os
DATA_PATH = Path(os.getenv("DATA_PATH"))
import pyedflib
import numpy as np

In [3]:
edf_path = str(DATA_PATH / "tueg/aaaaawey_s001_t001.edf")
f = pyedflib.EdfReader(edf_path)

In [9]:
# Check num of channels
print(f.signals_in_file)
# Check labels of channels
print(f.getSignalLabels())

31
['EEG FP1-REF', 'EEG FP2-REF', 'EEG F3-REF', 'EEG F4-REF', 'EEG C3-REF', 'EEG C4-REF', 'EEG P3-REF', 'EEG P4-REF', 'EEG O1-REF', 'EEG O2-REF', 'EEG F7-REF', 'EEG F8-REF', 'EEG T3-REF', 'EEG T4-REF', 'EEG T5-REF', 'EEG T6-REF', 'EEG A1-REF', 'EEG A2-REF', 'EEG FZ-REF', 'EEG CZ-REF', 'EEG PZ-REF', 'EEG ROC-REF', 'EEG LOC-REF', 'EEG EKG1-REF', 'EMG-REF', 'EEG T1-REF', 'EEG T2-REF', 'PHOTIC-REF', 'IBI', 'BURSTS', 'SUPPR']


In [10]:
# Frequencies of channels
f.getSampleFrequencies()

array([250., 250., 250., 250., 250., 250., 250., 250., 250., 250., 250.,
       250., 250., 250., 250., 250., 250., 250., 250., 250., 250., 250.,
       250., 250., 250., 250., 250., 250.,   1.,   1.,   1.])

In [11]:
# Number of samples for each channel
f.getNSamples()

array([327750, 327750, 327750, 327750, 327750, 327750, 327750, 327750,
       327750, 327750, 327750, 327750, 327750, 327750, 327750, 327750,
       327750, 327750, 327750, 327750, 327750, 327750, 327750, 327750,
       327750, 327750, 327750, 327750,   1311,   1311,   1311])

In [14]:
f.readSignal(0)

array([5.64924754e+00, 7.66438856e-01, 2.13972880e+00, ...,
       3.50000000e-03, 3.50000000e-03, 3.50000000e-03])